# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [1]:
# import modules
import math
from tqdm.autonotebook import tqdm
from collections import Counter


C:\Users\woute\AppData\Local\Temp\ipykernel_7052\964927310.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [2]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [3]:
def load_rules(filename: str) -> list[list[int]]:
    with open(filename) as f:
        raw_rules = f.read().splitlines()

    rules = [list(map(int, clause.split()))[:-1] for clause in raw_rules[1:]]
    return rules

def load_sudoku(filename: str) -> tuple[list[list[list[int | str]]], int]:
    with open(filename) as f:
        sudoku = f.read().splitlines()

    puzzles: list[list[list[int | str]]] = []
    
    puzzledim: int = math.sqrt(len(sudoku[0]))
    assert puzzledim.is_integer(), "The sudoku is not square"
    puzzledim = int(puzzledim)
    print(f"puzzledim: {puzzledim}")

    for puzzle in sudoku:
        newpuzzle = []
        for r in range(puzzledim):
            row = []
            for c in range(puzzledim):
                val = puzzle[r * puzzledim + c]
                if val == '.': 
                    val = 0
                elif puzzledim == 16 and val.isalpha():
                    # encode 1 to g values to 1 to 16
                    val = int(ord(val) - 55)
                else:
                    val = int(val)
                row.append(val)
            newpuzzle.append(row)
        puzzles.append(newpuzzle)
    return puzzles, puzzledim

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [4]:
def printinclause(clauses: list[list[int]], literal: int, extra: str) -> None:
    for clause in clauses:
        for lit in clause:
            if lit//10 == literal:
                print('In clause!', end=extra + '\n')
                return

def propagate(clauses: list[list[int]], literal: int) -> list[list[int]]:
    """Propagates a literal and simplifies the clauses."""
    new_clauses = []
    for clause in clauses:
        if literal in clause:
            continue  # Clause is satisfied, so skip it
        new_clause = [l for l in clause if l != -literal]
        # If new_clause is empty, we should add it as an empty clause to signal unsatisfiability.
        new_clauses.append(new_clause)
    return new_clauses

def single_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]] | None:
    """Check for clauses that have only one literal in them, if so we can assign that literal to True and remove the clause.

    Args:
        clauses (list[list[int]]): List of clauses in dimacs format
        assignment (dict[int, bool]): Dictionary of assignments

    Returns:
        list[list[int]] | None: Simplified clauses
    """
    changed = True
    while changed:
        changed = False
        unit_clauses = [c for c in clauses if len(c) == 1]
        for unit in unit_clauses:
            literal = unit[0]
            assignment[abs(literal)] = literal > 0
            clauses = propagate(clauses, literal)
            changed = True
    return clauses

def pure_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]]:
    """Check for pure literals, meaning that if one literal is in the clauses but it's negative is never we can assign it.
    Turns out this isn't used often for sudoku and mainly slows down the solver.

    Args:
        clauses (list[list[int]]): Lists of clauses in dimacs format
        assignment (dict[int, bool]): Dictionary of assignments

    Returns:
        list[list[int]]: Simplified clauses
    """
    while True:
        literals = {lit for clause in clauses for lit in clause}
        pure_literals = [l for l in literals if -l not in literals]
        if not pure_literals:
            break
        for literal in pure_literals:
            assignment[abs(literal)] = literal > 0
            # Remove all clauses containing this pure literal
            clauses = [clause for clause in clauses if literal not in clause]
    return clauses


def choose_literal(clauses: list[list[int]], assignment: dict[int, bool], heuristic) -> int | None:
    """
    Chooses the next literal to assign based on the given heuristic.

    Args:
        clauses (list[list[int]]): List of clauses in DIMACS format.
        assignment (dict[int, bool]): Dictionary of current variable assignments.
        heuristic (function): Function to apply the heuristic logic.

    Returns:
        int | None: The chosen literal, or None if no unassigned literals remain.
    """
    unassigned = {abs(lit) for clause in clauses for lit in clause if abs(lit) not in assignment}
    if not unassigned:
        return None
    return heuristic(clauses, unassigned)


**Heuristics** - Two different heuristics of our choice

In [5]:
# Most frequent literal
import random

def random_literal(clauses: list[list[int]], unassigned: set[int]) -> int:
    return random.choice(list(unassigned))

In [ ]:
import heapq

# heuristic two
class VSIDS:
    def __init__(self, clauses: list[list[int]], alpha: float = 0.95) -> None:
        self.activities = {}
        self.literals = set()
        self.alpha = alpha
        self.decay_factor = 1.0

        # Initialize the counts
        for clause in clauses:
            for lit in clause:
                if lit not in self.activities:
                    self.activities[lit] = 0
                    self.literals.add(lit)

    def __call__(self, clauses: list[list[int]], unassigned: set[int]) -> int:
        # Check the available literals still in the clauses
        available = {lit for clause in clauses for lit in clause}
        
        # Choose the literal with the highest activity
        max_activity = -1
        best_literal = None
        for lit in available:
            if self.activities[lit] > max_activity:
                max_activity = self.activities[lit]
                best_literal = lit
        
        return best_literal

    def update(self, literal: int) -> None:
        # Increase the activity of the literal
        self.activities[literal] += self.decay_factor

    def decay(self) -> None:
        # Decay all activities, not used currently as it seemed to slow down the program
        self.decay_factor *= self.alpha
        for lit in self.activities:
            self.activities[lit] *= self.alpha
            

**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [7]:
def print_ass(assignment: dict) -> None:
    for key in assignment.keys():
        dimac = key if assignment[key] else -key
        print(dimac, end=' ')
    print('')

def dpll(clauses: list[list[int]], assignment: dict[int, bool], heuristic: callable, pbar: tqdm, history: list[int] = [0]) -> dict[int, bool] | None:
    """DPLL algorithm to solve logic problems and stuff.

    Args:
        clauses (list[list[int]]): List of clauses in dimacs format
        assignment (dict[int, bool]): Dictionary with variables that have a value assigned to them
        pbar (tqdm): Tqdm progress bar, to keep track of brrrrrr
        history (list[int], optional): What variables it has checked. Defaults to [0].

    Returns:
        dict[int, bool] | None: Returns solution if possible, else None
    """
    # print("DPLL Call with History:", history)
    # Apply single literal rule

    clauses = single_literal(clauses, assignment)

    if clauses is None:
        # if using vsids update activities
        if type(heuristic) == VSIDS:
            heuristic.update(history[-1])
        return None  # conflict
    if not clauses:
        return assignment  # all clauses satisfied

    # Apply pure literal rule
    clauses = pure_literal(clauses, assignment)
    if not clauses:
        return assignment  # all clauses satisfied

    # Select an unassigned literal -> old part before heuristics
    # literal = next((lit for clause in clauses for lit in clause if abs(lit) not in assignment), None)
    # if literal is None:
    #     return None  # assignment is complete
    # literal = abs(literal)

    #new for heuristics
    literal = choose_literal(clauses, assignment, heuristic=heuristic)
    if literal is None: # conflict
        if type(heuristic) == VSIDS:
            heuristic.update(history[-1])
        return None
    literal = abs(literal)
    
    pbar.update(1)
    # Check both true and false assignments
    for booly in [True, False]:
        updated_assignment = assignment.copy()
        updated_assignment[abs(literal)] = booly
        updated_clauses = propagate(clauses, literal if booly else -literal)
        
        result = dpll(updated_clauses, updated_assignment, heuristic, pbar, history + [literal if booly else -literal])
        if result:  # If a satisfying assignment is found
            return result
        
    # conflict
    if type(heuristic) == VSIDS:
            heuristic.update(history[-1])
    return None  # backtrack

*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [ ]:
def solve_sudoku(rules: list[list[int]], puzzle: list[list[int]], size: int, vsids: bool = False) -> list[list[int]] | None:
    """
    Solves a Sudoku puzzle using given rules and an initial puzzle setup.

    Args:
        rules (list[list[int]]): A list of rules in CNF dimacs form for Sudoku constraints.
        puzzle (list[list[int]]): A 9x9 grid representing the puzzle, with `0` for empty cells.
        size (int): The size of the Sudoku puzzle (e.g. 9 for a 9x9 puzzle).

    Returns:
        list[list[int]] | None: A solved 9x9 Sudoku grid if a solution exists, otherwise None.
    """
    assignment = {} # the sudoku
    amt_unknown = 0
    rcmult = 17 if size == 16 else 10
    for i, row in enumerate(puzzle):
        for j, value in enumerate(row):
            if value != 0:  # if 0, cell is empty, if not 0, it is a pre-filled cell
                # Make the value in row i, column j True if it is the value from the sudoku, all other numbers in that place False
                varnum = (i + 1) * (rcmult**2) + (j + 1) * rcmult + value
                assignment[varnum] = True
                rules = propagate(rules, varnum)
                for k in range(1, size+1):
                    if k != value:
                        assignment[(i + 1) * 100 + (j + 1) * 10 + k] = False
            else:
                amt_unknown += 1
    
    if vsids:
        heuristic = VSIDS(rules)
    
    with tqdm(total=amt_unknown * size,desc='Trying to find logic solution') as pbar:
        solution = dpll(rules, assignment, heuristic if vsids else random_literal, pbar)
    if not solution:
        return None
     
    # parse the solution back to a 9x9 grid
    result = [[0] * 9 for _ in range(9)]
    for var, value in solution.items():
        if value:  # Ensure var corresponds to a Sudoku cell (1-9 for each row, column)
            i = (var // 100) - 1
            j = ((var // 10) % 10) - 1
            num = var % 10
            result[i][j] = num
    return result

**Test Sudoku** - Try and see if it works

In [9]:
load_sudoku('./4x4_easy.txt')[1]

puzzledim: 4


4

In [10]:
# Load the Sudoku rules and the puzzle
filename = './sudoku-rules-16x16.txt'
rules = load_rules(filename)
# print(rules)

# Load the puzzles from the file and select the second puzzle (index 1)
puzzles, size = load_sudoku('./16x16.txt')
puzzle = puzzles[0]
print(puzzle)

heuristic = random_literal

# Solve the Sudoku puzzle
solution = solve_sudoku(rules, puzzle, size, vsids=True)

print('puzzle:     ', puzzle)
print('solution:   ', solution)

# If a solution is found, print the solved Sudoku grid
if solution:
    for row in solution:
        print(row)
        if 0 in row:
            print("PANIEK")
else:
    print("No solution found")
    # 263 385 517 566 647

puzzledim: 16
[[1, 0, 13, 0, 0, 0, 0, 4, 0, 10, 5, 8, 0, 0, 0, 0], [0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 16], [0, 2, 0, 7, 6, 0, 16, 11, 15, 0, 0, 4, 0, 0, 0, 0], [3, 9, 15, 0, 1, 10, 0, 13, 7, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 6, 0, 3, 1, 0, 0, 0, 11, 0, 5, 8, 0, 12], [8, 12, 7, 14, 0, 6, 9, 0, 0, 15, 0, 0, 0, 0, 0, 13], [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 10, 0, 16, 8, 12, 0, 0, 0, 0, 7, 14, 0, 1, 4], [2, 6, 0, 16, 4, 0, 0, 0, 0, 5, 7, 15, 0, 10, 0, 0], [0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0], [15, 0, 0, 0, 0, 0, 11, 0, 0, 3, 10, 0, 4, 2, 14, 1], [10, 0, 4, 12, 0, 5, 0, 0, 0, 14, 6, 0, 7, 0, 3, 0], [0, 0, 0, 0, 0, 0, 0, 3, 13, 0, 12, 5, 0, 7, 11, 2], [0, 0, 0, 0, 9, 0, 0, 1, 16, 11, 0, 6, 3, 0, 4, 0], [12, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0], [0, 0, 0, 0, 8, 15, 13, 0, 3, 0, 0, 0, 0, 9, 0, 14]]


Trying to find logic solution:   0%|          | 0/2528 [00:00<?, ?it/s]

puzzle:      [[1, 0, 13, 0, 0, 0, 0, 4, 0, 10, 5, 8, 0, 0, 0, 0], [0, 14, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 0, 0, 0, 16], [0, 2, 0, 7, 6, 0, 16, 11, 15, 0, 0, 4, 0, 0, 0, 0], [3, 9, 15, 0, 1, 10, 0, 13, 7, 0, 0, 0, 0, 0, 0, 0], [0, 4, 0, 6, 0, 3, 1, 0, 0, 0, 11, 0, 5, 8, 0, 12], [8, 12, 7, 14, 0, 6, 9, 0, 0, 15, 0, 0, 0, 0, 0, 13], [0, 0, 0, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0], [0, 0, 10, 0, 16, 8, 12, 0, 0, 0, 0, 7, 14, 0, 1, 4], [2, 6, 0, 16, 4, 0, 0, 0, 0, 5, 7, 15, 0, 10, 0, 0], [0, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0], [15, 0, 0, 0, 0, 0, 11, 0, 0, 3, 10, 0, 4, 2, 14, 1], [10, 0, 4, 12, 0, 5, 0, 0, 0, 14, 6, 0, 7, 0, 3, 0], [0, 0, 0, 0, 0, 0, 0, 3, 13, 0, 12, 5, 0, 7, 11, 2], [0, 0, 0, 0, 9, 0, 0, 1, 16, 11, 0, 6, 3, 0, 4, 0], [12, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0], [0, 0, 0, 0, 8, 15, 13, 0, 3, 0, 0, 0, 0, 9, 0, 14]]
solution:    [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 9, 0, 0, 8, 0, 6, 3], [1, 0, 0, 0, 1, 0, 0, 0, 0], [6, 0, 0, 1, 9